# Onderzoek vragen

Wij hebben de volgende onderzoek vragen bedacht.

1. In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?
2. (Bonus) Kan men op basis van eigenschappen van eerder gereviewde wijnen voorspellen wat een recensent van een wijn zal vinden.
3. Welk effect heeft weer over de jaren heen effect op de wijn score die reviewers geven in punten?
4. Kan je op basis van score en prijs welke regio de wijn zijn oorsprong.


# Data verkening 
Als eers hebben wij het csv naar een exel bestand omgezet en de data bekeken. Hier uit ondeketen wij de volgende punten.

Wij er achter gekomen dat het moeilijk is om een wijn te identificeren. Zo kan een wijn uit hetzelfde jaar met dezelfde naam gereviewed door dezelfde persoon uit de zelfde streek andere chemische eigenschappen hebben. Dit roept vragen op. Namelijk zijn de wijnen hetzelfde en zo ja. Klopt de meting van de chemische eigenschappen dan wel. Wij hebben besloten er van uit te gaan dat wij de chemische eigenschappen kunnen vertrouwen. Dit omdat wij anders dusdanig beperkt zijn dat wij geen interessante correlaties kunnen maken en de opdracht dus niet voldaan kan worden. Wij weten dus niet in hoeverre de namen correct zijn. Dit kan voor problemen zorgen bij ML. 
Jaartal is niet specifiek aangegeven maar kan wel zeer bruikbaar zijn. Naar verkenning zijn wij er achter gekomen dat een titel wel vaak een jaar aangeeft. Wij hebben toen code geschreven om te kijken welke titels geen jaar bevat dit bleken er 4 te zijn.
Na verkenning zijn wij er achter gekomen dat provincies niet altijd bestaande provincies zijn maar dat er ook regio’s en onbruikbare waardes tussen zitten. Dit zorgt er dus voor dat wij niet soepel weerdata voor die regio kunnen opvragen en maakt een correlatie zoeken tussen weer in een regio en de  qualiteit van de wijn erg moeilijk Daarom kiezen wij ervoor om data van de hoofdstad te gebruiken i.p.v. data van verschillende plaatsen. De data van de hoofdstad (Lissabon) hebben we gevonden op www7.ncdc.noaa.gov/ dit is National Climatic Data Center van de United states department of commerce. Hier hebben wij de data van Lissabon van 1990 tot 2018 kunnen vinden.


In [ ]:
# importeer nodige libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

__INTERNE RED WINES DATASET__

In [ ]:
# lees het bestand in
redwines = pd.read_csv("redwine.csv", delimiter=";", encoding='iso-8859-1')

# verwijder onnodige kolommen
redwines.drop(['country','variety','id'],axis=1, inplace=True)
redwines.dropna(inplace=True)

# voeg jaartal toe als aparte kolom, en verwijder nan rijen voor jaren (2465 naar 2461)
redwines['year'] = redwines['title'].str.extract(r"([1][9][9]\d|[2][0][0,1,2]\d)")
redwines = redwines[redwines['year'].notnull()]
redwines['year'] = redwines['year'].astype(int)
redwines['count'] = 1
#redwines = redwines[pd.to_numeric(redwines['citric acid'], errors='coerce').notnull()]
#redwines['citric acid'] = redwines['citric acid'].str.replace(',','.').astype(float)

# lees tabel statistieken uit
redwines.describe()

__EXTERNE WEER DATASET__

In [ ]:
# lees de externe dataset in
weather_conditions = pd.read_csv("weatherdata_lisbon.csv", delimiter=";", encoding='iso-8859-1')

# drop onbruikbare rijen en hernoem kolommen
weather_conditions.drop(['STN---','Unnamed: 5','Unnamed: 7','GUST','PRCP','SNDP','VISIB'], axis=1,inplace=True)
weather_conditions.dropna(inplace=True)
weather_conditions.rename(columns={'YEARMODA': 'Year','DEWP':'Dewpoint Temperature','TEMP': 'Temperature','MAX':'Max Temperature','MIN':'Min Temperature','WDSP':'Windspeed','MXSPD':'Max Windspeed'},inplace=True)

# cleanup and transform data types
# also convert fahrenheit to celsius
weather_conditions['Max Temperature'] = (weather_conditions['Max Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Min Temperature'] = (weather_conditions['Min Temperature'].str.replace(',','.').str.rstrip("*").astype(float) - 32) / 1.8
weather_conditions['Temperature'] = (weather_conditions['Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Dewpoint Temperature'] = (weather_conditions['Dewpoint Temperature'].str.replace(',','.').astype(float) - 32) / 1.8
weather_conditions['Windspeed'] = weather_conditions['Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Max Windspeed'] = weather_conditions['Max Windspeed'].str.replace(',','.').astype(float)
weather_conditions['Year']= pd.to_datetime(weather_conditions['Year'].astype(str), format='%Y-%m-%d')

# FRSHTT = Frost, Rain, Snow, Hail, Thunder, Thornado
weather_conditions['FRSHTT'] = weather_conditions['FRSHTT'].apply(lambda x: '{0:0>6}'.format(x))
weather_conditions['Frost'] = weather_conditions['FRSHTT'].str[0].astype(float)
weather_conditions['Rain'] = weather_conditions['FRSHTT'].str[1].astype(float)
weather_conditions['Snow'] = weather_conditions['FRSHTT'].str[2].astype(float)
weather_conditions['Hail'] = weather_conditions['FRSHTT'].str[3].astype(float)
weather_conditions['Thunder'] = weather_conditions['FRSHTT'].str[4].astype(float)
weather_conditions['Tornado'] = weather_conditions['FRSHTT'].str[5].astype(float)
weather_conditions.drop('FRSHTT',axis=1,inplace=True)

# verwijder uitschieters
weather_conditions = weather_conditions[weather_conditions['Min Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Temperature'] < 50]
weather_conditions = weather_conditions[weather_conditions['Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Max Windspeed'] < 50]
weather_conditions = weather_conditions[weather_conditions['Dewpoint Temperature'] < 100]

# bereken gemiddelde eigenschappen per jaar
weather_conditions['Count'] = 1
average_weather_conditions_peryear = weather_conditions.groupby(weather_conditions['Year'].map(lambda x: x.year)).sum()
average_weather_conditions_peryear = average_weather_conditions_peryear[['Temperature','Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder','Tornado']].div(average_weather_conditions_peryear['Count'], axis=0)
average_weather_conditions_peryear.drop(pd.Int64Index([2018]), inplace=True) # 2018 heeft maar 1 meting

# lees tabel statistieken uit
average_weather_conditions_peryear.describe()

__MERGING EXTERNE EN INTERNE DATASETS__

In [ ]:
redwines_with_yeartemperatures = redwines.merge(average_weather_conditions_peryear, left_on='year', right_on='Year')

__DATA ANALYSES AND GRAPHS__

In [ ]:
# corralation tabel voor dataset redwines
plt.figure(figsize=(16,12))
correlation_matrix = redwines_with_yeartemperatures.drop(['description','title','Tornado','count'],axis=1).corr()
sns.heatmap(correlation_matrix, annot=True, cmap=plt.cm.Reds)
plt.show()

Het is belangrijk om inzicht te hebben in hoe de punten zijn verdeeld. Na een korte analyse kwamen we erachter dat alle punten zijn verdeeld tussen 80 en 100. Daarmee is een wijn van 80 punten opeens de slechtst beoordeelde wijn terwijl het lijkt alsof dit een redelijk hoge score is. In onderstaande bar plot is te zien hoe de punten zijn verdeeld 

In [ ]:
# punt verdeling
points = redwines.groupby('points').sum()
plt.figure(figsize=(16,5))
plt.xticks(points.index)
plt.bar(points.index, points['count'])
plt.xlabel('Points')
plt.ylabel('Count')
plt.show()

Wij hebben ook besloten om een paar box plotten te maken van specifieken waarden om zo te zien hoe die zijn verdeeld
Hier uit is te zien dat de pH relatief dicht bij elkaar licht en dat er niet veel waarden zijn die erveel afwijken

In [ ]:
boxplot = redwines.boxplot('pH')

Hier uit is te zien dat het suiker gehalte veel waardes zijn die ver afwijken. Intresant genoeg zijn al deze waarden hoger dan de mediaan en zijn er weinig waarden lager

In [ ]:
boxplot = redwines.boxplot('residual sugar')

We vroegen ons als groepje ook af wat de correlatie is tussen wijn prijs en gegeven punten. Je zou zeggen dat de duurdere wijnen ook beter beoordeeld moeten zijn. Het aantal punten zou dus moeten oplopen naarmate de prijs hoger wordt. Dit bleek ook enigzins het geval zoals in de onderstaande scatter plot te zien is. Het is soms ook lastig te zeggen aangezien het onderlinge verschil tussen gegeven punten niet erg hoog is. Er is per puntenaantal een gemiddeld van de prijs genomen, en dit gemiddelde wordt getoond per puntenaantal.

In [ ]:
# Points comparison to average price
avg = redwines.groupby('points').sum()
avg['avg_price_per_point'] = avg['price'] / avg['count']
plt.figure(figsize=(16,10))
plt.xticks(avg.index)
plt.bar(avg.index, avg['avg_price_per_point'])
plt.xlabel('Points')
plt.ylabel('Average price')
plt.show()

Ook dachten we dat er misschien een verband tussen goedkope wijn en veel suiker zou zijn, i.v.m. het feit dat suiker een goedkope grondstof is. Dit bleek echter minder uit te maken dan we eerst dachten. Er is geen overduidelijk verband te vinden tussen lagere wijnprijzen en hogere suikergehaltes zoals in de onderstaande scatter plot te zien is.

In [ ]:
# Sugar comparison to price
plt.figure(figsize=(16,5))
plt.scatter(redwines['residual sugar'], redwines['price'])
plt.ylabel('Price')
plt.xlabel('Residual sugar')
plt.show()

In [ ]:
# Sugar increase over years
avg = redwines.groupby('year').sum()
avg['avg_increase_substance'] = avg['residual sugar'] / avg['count']
plt.figure(figsize=(16,5))
plt.plot(avg.index, avg['avg_increase_substance'])
plt.ylabel('Substance')
plt.xlabel('Year')
plt.show()

In [ ]:
# Average scores per taster
avg = redwines.groupby('taster_name').sum()
avg['avg_taster_given_points'] = avg['points'] / avg['count']
plt.figure(figsize=(16,5))
axes = plt.gca()
axes.set_ylim([80,95])
plt.bar(avg.index, avg['avg_taster_given_points'])
plt.ylabel('Average points')
plt.xlabel('Taster')
plt.xticks(avg.index, rotation='vertical')
plt.show()

Ook hebben wij gekeken naar de correlatie punten en jaar. Goed is wel om te vermelde dat er weinig wijnen uit het jaar 1996 kwamen. Dus die scoren is gebaseerd op een zeer kleine testgroep. 

In [ ]:
# voeg jaartal toe als aparte kolom, en verwijder nan rijen voor jaren (2465 naar 2461)

redwines['year'] = redwines['title'].str.extract(r"([1][9][9]\d|[2][0][0,1,2]\d)")

redwines = redwines[redwines['year'].notnull()]

# drop rows with nan value in it

redwines.dropna(inplace=True)

# adds how many wines are tested in a year

redwines['count'] = 1

#average of points given per year 

sum_points_per_year = redwines.groupby('year').sum()

sum_points_per_year['avg_points_per_year'] = sum_points_per_year['points'] / sum_points_per_year['count']


#avg points given per year
plt.plot(sum_points_per_year.index, sum_points_per_year['avg_points_per_year'])
plt.xticks(sum_points_per_year.index, sum_points_per_year.index, rotation='vertical')
plt.title('avg points per year of wines')
plt.xlabel('year')
plt.ylabel('points')
plt.show()

__CREATING THE MODEL FOR CORRELATION BETWEEN POINTS AND AVERAGE YEARLY WEATHER CONDITIONS__

In [ ]:
# normalize data
#x = redwines_with_yeartemperatures.drop(['description','title','designation','taster_name','province','winery'],axis=1).values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#redwines_with_yeartemperatures = pd.DataFrame(x_scaled)

# ['Temperature', 'Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder']
# dependent and independent columns
dependent = redwines_with_yeartemperatures['points']
independent = redwines_with_yeartemperatures[['Temperature', 'Dewpoint Temperature','Windspeed','Max Windspeed','Max Temperature','Min Temperature','Frost','Rain','Snow','Hail','Thunder']]

# test and train set
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.05)

# setting up and training model
model = LogisticRegression()
model.fit(X_train, y_train)

# testing and calculating accuracy
#print(model.predict(X_test))
print(model.score(X_test,y_test))

# LinearRegression 8%
# DecisionTreeClassifier 12%
# LogisticRegression 15%
# BayesianRidge 6%
# SVR 5%
# DecisionTreeRegressor 7%
# Ridge 5%

__CREATING THE MODEL FOR CORRELATION BETWEEN POINTS AND CHEMICAL PROPERTIES__

In [ ]:
# ['fixed acidity', 'volatile acidity','citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide','total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
# ["fixed acidity", "volatile acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "pH", "sulphates"]
# dependent and independent columns
dependent = redwines_with_yeartemperatures["points"]
#independent = redwines_with_yeartemperatures[["fixed acidity", "volatile acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "pH", "sulphates"]]
independent = redwines_with_yeartemperatures[["volatile acidity","total sulfur dioxide"]]
# test and train set
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, random_state=0, test_size=0.2)

# setting up and training model
model = LogisticRegression()
model.fit(X_train, y_train)

# testing and calculating accuracy
#print(model.predict([[13,25]]))
print(model.score(X_test,y_test))

# LinearRegression 8%
# DecisionTreeClassifier 12%
# LogisticRegression 15%
# BayesianRidge 6%
# SVR 5%
# DecisionTreeRegressor 7%
# Ridge 5%